In [1]:
### Benchmark Fine-Tuned NLLB-200 Model for English-Chinese Translation using BLEU

In [2]:
import sacrebleu
import ctranslate2
from datasets import load_dataset
from tqdm import tqdm

import sys
import os
sys.path.append(os.getcwd())  # Adds current notebook's directory

from text_to_text import translate_text

/home/kayleexie/miniconda3/envs/cs329e/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
dataset = load_dataset("opus100", "en-zh")  # 1M+ sentence pairs

In [4]:
def prepare_batch(batch):
    src_texts = batch['translation']['en']
    tgt_texts = batch['translation']['zh']
    preds = translate_text(src_texts, "English", "Chinese (Simplified)", beam_size=5)
    return preds, tgt_texts

In [15]:
predictions = []
references = []

In [ ]:
for sample in tqdm(dataset["test"].select(range(2)), desc="Translating"):
    pred, ref = prepare_batch(sample)
    # print(pred)
    # print(type(pred))
    # print(ref)
    # print(type(ref))
    predictions.append(pred)
    references.append(ref)  # Each reference must be a list of one or more refs

Translating:  50%|█████     | 1/2 [00:29<00:29, 29.14s/it]

全球行动计划协调办公室目前通过比利时的财政支持协助埃及,尼日利亚,坦桑尼亚联合共和国,斯里兰卡和也门制定试点国家行动方案,以保护海洋环境免受陆地活动的影响.
<class 'str'>
9. 《全球行动纲领》协调处得到比利时的财政支持，目前正帮助埃及、尼日利亚、坦桑尼亚联合共和国、斯里兰卡和也门制订试行的保护海洋环境免受陆地活动影响的国家行动方案。
<class 'str'>


Translating: 100%|██████████| 2/2 [00:43<00:00, 21.72s/it]

该论坛在亚洲和非洲私人公司之间签订了超过2400万美元的商业合同.
<class 'str'>
该论坛达成的亚洲和非洲国家私营公司间的商业合同超过了2,400万美元。
<class 'str'>


In [ ]:
# print(predictions)

['全球行动计划协调办公室目前通过比利时的财政支持协助埃及,尼日利亚,坦桑尼亚联合共和国,斯里兰卡和也门制定试点国家行动方案,以保护海洋环境免受陆地活动的影响.', '该论坛在亚洲和非洲私人公司之间签订了超过2400万美元的商业合同.']


In [ ]:
# print(references)

['9. 《全球行动纲领》协调处得到比利时的财政支持，目前正帮助埃及、尼日利亚、坦桑尼亚联合共和国、斯里兰卡和也门制订试行的保护海洋环境免受陆地活动影响的国家行动方案。', '该论坛达成的亚洲和非洲国家私营公司间的商业合同超过了2,400万美元。']


In [19]:
bleu = sacrebleu.corpus_bleu(predictions, [references])
print(f"\n📈 BLEU Score: {bleu.score:.5f}")


📈 BLEU Score: 3.90492


In [ ]:
for i in range(2):
    print(f"\nExample {i+1}:")
    print(f"English Input: {dataset['test'][i]['translation']['en']}")
    print(f"Reference Chinese: {references[i]}")
    print(f"Model Prediction: {predictions[i]}")


Example 1:
English Input: The Global Programme of Action Coordination Office, with the financial support of Belgium, is currently assisting Egypt, Nigeria, United Republic of Tanzania, Sri Lanka and Yemen to develop pilot national programmes of action for the protection of the marine environment from land-based activities.
Reference Chinese: 9. 《全球行动纲领》协调处得到比利时的财政支持，目前正帮助埃及、尼日利亚、坦桑尼亚联合共和国、斯里兰卡和也门制订试行的保护海洋环境免受陆地活动影响的国家行动方案。
Model Prediction: 全球行动计划协调办公室目前通过比利时的财政支持协助埃及,尼日利亚,坦桑尼亚联合共和国,斯里兰卡和也门制定试点国家行动方案,以保护海洋环境免受陆地活动的影响.

Example 2:
English Input: The forum produced business contracts amounting to more than $24 million between Asian and African private companies.
Reference Chinese: 该论坛达成的亚洲和非洲国家私营公司间的商业合同超过了2,400万美元。
Model Prediction: 该论坛在亚洲和非洲私人公司之间签订了超过2400万美元的商业合同.
